# HW 4 CS 334
Hanna Meinikheim

9-10-2021

Logistic Regression

# Logistic Regression
---
Is it going to rain in Boise today? Create a Jupyter Notebook at that performs Logistic Regression using scikit-learn to predict the answer to this yes/no question.

- [ ] **Find some data.** You’re looking for a data set that has historic weather data for Boise, Idaho. Search for one yourself, that’s part of the assignment! There are many websites that offer APIs or searches that allow free downloads of a given time range and location. Select at least one year (maybe more) of recent weather data. A good data set will have things like temperature, barometric pressure, wind speed, etc. And of course, it must have some indication about whether it rained that day, usually in the form of inches of precipitation.
- [ ] **Pull this data into your project.** Download the data, and import it into your project as a file. Perhaps using Pandas and import it as a CSV file? You may have to create a new column indicating the label: `0 = no rain, 1 = rain`. This column is an important part of the training process. It’s not a feature, it’s a label for that given vector.
- [ ] **Use Logistic Regression.** You’re trying to answer the question, “Is it going to rain today in Boise?” Train a model that tries to predict if it’s going to rain on a given day. The “day” is a vector/row/array/sample of given features that you feed into the model to get a prediction. Remember, you need the same number of features (columns) for a given prediction that you used to train the model.
- [ ] **How good is your model?** Is it better than a wild guess? Document your findings in the notebook.
- [ ] **Push Jupyter Notebook to github**

### Finding the data
- I found out that Weather Underground no longer gives out free weather API keys as part of their program. [Weather Underground API GONE?](https://forum.pvoutput.org/t/weather-underground-api-gone/576)
- I submitted an order to NOAA and got the requested data. I just have to upload it to the Jupyter Notebook.

In [ ]:
#import pandas
import pandas as pd

col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']

# load dataset
patients = pd.read_csv("diabetes.csv", header=None, names=col_names, skiprows=[0])